In [3]:
%load_ext autoreload
%autoreload 2
import numpy as np
import matplotlib.pyplot as plt
from coffea import util
from coffea.processor import accumulate
import os, sys
import re
import pandas as pd
import hist
import matplotlib as mpl
import mplhep
import utils
from pathlib import Path

sys.path.append("../../../analysisTools/")
import plotTools as ptools

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
def add_signal_info_to_df(df):
    m1_list = []
    delta_list = []
    ctau_list = []
    
    for point in df.index.values:
        sig_dict = ptools.signalPoint(point)
        m1_list.append(sig_dict['m1'])
        delta_list.append(sig_dict['delta'])
        ctau_list.append(sig_dict['ctau'])
    
    df['m1'] = m1_list
    df['delta'] = delta_list
    df['ctau'] = ctau_list
    
    df = df.sort_values(by=['m1']) # sort by m1

    return df

In [5]:
def save_df_to_csv(df, outdir, outname, isSignal = False):
    Path(outdir).mkdir(parents=True, exist_ok=True)

    if isSignal:
        df = add_signal_info_to_df(df)
    
    df.to_csv(f'{outdir}/{outname}.csv')

    print(f'Saved: {outdir}/{outname}.csv')

# SR Lxy-BDT-based trained with [5,50]GeV: run on all signal mass points [5, 100] GeV

## Background

In [6]:
prefix = './coffea/skimmed/'
suffix = '_SR_BDT-based_Lxy_trained_5to50'

In [17]:
# Background
bkg_files = {
    "QCD": "bkg_2018_QCD_TuneCP5_PSWeights", # No entry for QCD
    "Top": "bkg_2018_Top",
    "DY": "bkg_2018_DY",
    "Diboson": "bkg_2018_Diboson",
    "Triboson": "bkg_2018_Triboson",
    "W+jets": "bkg_2018_WJets",
    "Z+jets": "bkg_2018_ZJets",
}

bkg_histos = {}

for b in bkg_files.keys():
    bkg_histos[b] = util.load(f'{prefix}/{bkg_files[b]}{suffix}.coffea')[0]

In [18]:
# efficiency
pd.options.display.float_format = '{:,.6f}'.format
bkg_eff_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow')

bkg_eff_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
QCD,0.000002,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Top,0.000129,0.000013,0.000013,0.000008,0.000007,0.000007,0.000007,0.000007,0.000000
DY,0.000011,0.000001,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
Diboson,0.000637,0.000044,0.000044,0.000033,0.000031,0.000030,0.000030,0.000027,0.000000
Triboson,0.001933,0.000158,0.000158,0.000104,0.000092,0.000091,0.000091,0.000076,0.000000
W+jets,0.002310,0.000173,0.000173,0.000139,0.000132,0.000131,0.000131,0.000114,0.000000
Z+jets,0.006645,0.000398,0.000398,0.000325,0.000310,0.000309,0.000309,0.000292,0.000000
Total,0.000019,0.000001,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [19]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_BDT_Lxy-based_trained_5to50_Background_Efficiency'

df_to_save = bkg_eff_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/Efficiency//SR_BDT_Lxy-based_trained_5to50_Background_Efficiency.csv


In [20]:
# weighted event count
pd.options.display.float_format = '{:,.2f}'.format
bkg_cts_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_cts')

bkg_cts_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
QCD,"38,039.83","1,882.67","1,882.67","1,326.86","1,319.01","1,319.01","1,318.41",632.25,0.00
Top,"5,667.49",559.22,559.22,349.76,314.03,312.99,312.95,294.00,0.53
DY,"14,495.72",671.44,671.44,623.25,623.25,623.25,623.25,455.65,0.00
Diboson,"4,424.20",307.91,307.91,227.24,213.36,211.27,210.33,189.35,0.58
Triboson,53.00,4.32,4.32,2.85,2.51,2.51,2.50,2.09,0.00
W+jets,"227,212.60","17,009.81","17,009.81","13,645.63","12,937.21","12,911.49","12,907.15","11,204.39",20.31
Z+jets,"142,220.15","8,524.55","8,524.55","6,957.67","6,625.48","6,613.11","6,612.30","6,256.76",7.89
Total,"432,113.00","28,959.92","28,959.92","23,133.27","22,034.86","21,993.63","21,986.91","19,034.50",29.31


In [21]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_BDT_Lxy-based_trained_5to50_EventCount_Weighted'

df_to_save = bkg_cts_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDT_Lxy-based_trained_5to50_EventCount_Weighted.csv


In [22]:
# raw event count
pd.options.display.float_format = '{:,.0f}'.format
bkg_cts_raw_df = utils.get_bkg_cutflow_df(bkg_histos, 'cutflow_nevts')

bkg_cts_raw_df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
QCD,12311,381,381,160,159,159,158,3,0
Top,49818,4688,4688,3058,2348,2337,2336,2006,3
DY,205,13,13,11,11,11,11,9,0
Diboson,18067,1234,1234,915,860,852,848,767,2
Triboson,2257,170,170,116,106,105,104,92,0
W+jets,697657,48425,48425,37015,35472,35415,35377,29090,58
Z+jets,1281459,67427,67427,52627,50986,50934,50903,44480,60
Total,2061774,122338,122338,93902,89942,89813,89737,76447,123


In [23]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_BDT_Lxy-based_trained_5to50_Background_EventCount_Raw'

df_to_save = bkg_cts_raw_df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDT_Lxy-based_trained_5to50_Background_EventCount_Raw.csv


In [24]:
nBkg = bkg_cts_df.loc['Total'][-1]

print(f'Total number of backgrounds (weighted) after the last SR cut: {nBkg:.2f}')

Total number of backgrounds (weighted) after the last SR cut: 29.31


In [25]:
pd.options.display.float_format = '{:,.2f}'.format

bkg_composition = pd.concat([pd.DataFrame(bkg_cts_df['BDT']).transpose(), pd.DataFrame(bkg_cts_raw_df['BDT']).transpose()], ignore_index=True)

bkg_composition = bkg_composition.rename(index={0: 'Weighted Counts', 1: 'Raw Counts'})
bkg_composition

,QCD,Top,DY,Diboson,Triboson,W+jets,Z+jets,Total
Weighted Counts,0.00,0.53,0.00,0.58,0.00,20.31,7.89,29.31
Raw Counts,0.00,3.00,0.00,2.00,0.00,58.00,60.00,123.00


## Signal

In [26]:
# Signal
sig_histo = util.load(f'{prefix}/signal{suffix}.coffea')[0]

In [27]:
si = utils.get_signal_point_dict(sig_histo)

pd.options.display.float_format = '{:,.1f}'.format
si

,mchi,dmchi,ctau,m1,m2,delta,name
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,94.5,9.0,100.0,90.0,99.0,0.1,sig_2018_Mchi-94p5_dMchi-9p0_ctau-100
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,99.0,18.0,100.0,90.0,108.0,0.2,sig_2018_Mchi-99p0_dMchi-18p0_ctau-100
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,99.0,18.0,1.0,90.0,108.0,0.2,sig_2018_Mchi-99p0_dMchi-18p0_ctau-1
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,99.0,18.0,10.0,90.0,108.0,0.2,sig_2018_Mchi-99p0_dMchi-18p0_ctau-10
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,88.0,16.0,10.0,80.0,96.0,0.2,sig_2018_Mchi-88p0_dMchi-16p0_ctau-10
...,...,...,...,...,...,...,...
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,33.0,6.0,1.0,30.0,36.0,0.2,sig_2018_Mchi-33p0_dMchi-6p0_ctau-1
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,21.0,2.0,100.0,20.0,22.0,0.1,sig_2018_Mchi-21p0_dMchi-2p0_ctau-100
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.5,1.0,10.0,10.0,11.0,0.1,sig_2018_Mchi-10p5_dMchi-1p0_ctau-10
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,22.0,4.0,1.0,20.0,24.0,0.2,sig_2018_Mchi-22p0_dMchi-4p0_ctau-1


In [28]:
cutnames = utils.get_signal_list_of_cuts(sig_histo)

cutnames

['Preselections',
 '0 < n(jet) < 3 & n(good vertex) > 0',
 'computing BDT score',
 'No b-tagged jets',
 'Leading jet |eta| < 2.4',
 'Leading jet pT > 80 GeV',
 'dPhi(MET,leading jet) > 1.5',
 'dPhi(MET,all jets) > 0.75',
 'BDT']

### Efficiency

In [29]:
pd.options.display.float_format = '{:,.6f}'.format

In [30]:
eff_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow')

In [31]:
table = pd.DataFrame.from_dict(eff_cutflow, orient='index')
table.columns = cutnames

In [32]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,0.065041,0.015343,0.015343,0.012976,0.012527,0.012527,0.012527,0.012353,0.002566
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,0.066110,0.013011,0.013011,0.011131,0.010744,0.010743,0.010743,0.010641,0.003863
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,0.058112,0.011195,0.011195,0.009729,0.009420,0.009420,0.009420,0.009342,0.003545
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,0.049041,0.011230,0.011230,0.009552,0.009201,0.009201,0.009201,0.009076,0.003042
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,0.051428,0.009301,0.009301,0.007902,0.007672,0.007672,0.007672,0.007581,0.003051
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,0.043559,0.007791,0.007791,0.006622,0.006351,0.006351,0.006351,0.006279,0.002655
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,0.014484,0.001842,0.001842,0.001519,0.001421,0.001418,0.001418,0.001404,0.000602
sig_2018_Mchi-60p0_dMchi-20p0_ctau-1,0.029875,0.007027,0.007027,0.005808,0.005631,0.005631,0.005631,0.005523,0.001569
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,0.036318,0.006259,0.006259,0.005421,0.005217,0.005217,0.005217,0.005197,0.002363
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,0.033826,0.006982,0.006982,0.005961,0.005695,0.005695,0.005695,0.005651,0.003071


In [33]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_BDT_Lxy-based_trained_5to50_Signal_ctau-1_samples_Efficiency'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_BDT_Lxy-based_trained_5to50_Signal_ctau-1_samples_Efficiency.csv


In [34]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,0.060097,0.011533,0.011533,0.009790,0.009461,0.009457,0.009457,0.009230,0.002629
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,0.051928,0.009921,0.009921,0.008256,0.008034,0.008034,0.008034,0.007820,0.002916
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,0.058330,0.009363,0.009363,0.007922,0.007656,0.007656,0.007656,0.007545,0.002847
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,0.052317,0.008360,0.008360,0.007155,0.006897,0.006897,0.006897,0.006840,0.002557
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,0.045542,0.008151,0.008151,0.006929,0.006680,0.006676,0.006676,0.006519,0.002404
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,0.045232,0.006990,0.006990,0.006015,0.005827,0.005827,0.005827,0.005734,0.002340
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,0.039166,0.005699,0.005699,0.004893,0.004680,0.004676,0.004676,0.004619,0.001740
sig_2018_Mchi-60p0_dMchi-20p0_ctau-10,0.028663,0.005058,0.005058,0.004260,0.004136,0.004136,0.004136,0.003913,0.001443
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,0.010676,0.001022,0.001022,0.000847,0.000792,0.000792,0.000792,0.000771,0.000171
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,0.010930,0.001187,0.001187,0.000987,0.000935,0.000931,0.000931,0.000876,0.000266


In [35]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_BDT_Lxy-based_trained_5to50_Signal_ctau-10_samples_Efficiency'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_BDT_Lxy-based_trained_5to50_Signal_ctau-10_samples_Efficiency.csv


In [36]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,0.041559,0.004565,0.004565,0.003872,0.003775,0.003775,0.003775,0.003670,0.000883
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,0.042898,0.005171,0.005171,0.004361,0.004208,0.004206,0.004206,0.004008,0.000743
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,0.038143,0.004461,0.004461,0.003821,0.003680,0.003677,0.003677,0.003497,0.000827
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,0.036959,0.004105,0.004105,0.003461,0.003353,0.003349,0.003349,0.003245,0.000780
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,0.032554,0.003600,0.003600,0.003015,0.002870,0.002870,0.002870,0.002746,0.000652
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,0.032810,0.003467,0.003467,0.002935,0.002837,0.002837,0.002837,0.002743,0.000594
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,0.027190,0.002624,0.002624,0.002212,0.002100,0.002100,0.002100,0.002042,0.000490
sig_2018_Mchi-60p0_dMchi-20p0_ctau-100,0.021028,0.002322,0.002322,0.001971,0.001921,0.001921,0.001921,0.001762,0.000409
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.007414,0.000647,0.000647,0.000562,0.000528,0.000528,0.000528,0.000501,0.000048
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,0.007306,0.000596,0.000596,0.000504,0.000475,0.000475,0.000475,0.000457,0.000034


In [37]:
outdir = './csv/Cutflow/Efficiency/'
outname = 'SR_BDT_Lxy-based_trained_5to50_Signal_ctau-100_samples_Efficiency'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/Efficiency//SR_BDT_Lxy-based_trained_5to50_Signal_ctau-100_samples_Efficiency.csv


### Weighted Event count

In [38]:
pd.options.display.float_format = '{:,.2f}'.format

In [39]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')

In [40]:
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [41]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,51.58,12.17,12.17,10.29,9.93,9.93,9.93,9.80,2.03
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,"1,627.11",320.24,320.24,273.97,264.43,264.40,264.40,261.91,95.09
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,"2,122.81",408.93,408.93,355.40,344.10,344.10,344.10,341.27,129.49
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,90.58,20.74,20.74,17.64,16.99,16.99,16.99,16.76,5.62
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,"2,928.06",529.56,529.56,449.92,436.79,436.79,436.79,431.62,173.72
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,"4,153.77",742.93,742.93,631.48,605.60,605.60,605.60,598.76,253.16
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,349.57,288.31,269.62,268.98,268.98,266.40,114.19
sig_2018_Mchi-60p0_dMchi-20p0_ctau-1,5.68,1.33,1.33,1.10,1.07,1.07,1.07,1.05,0.30
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"6,562.28","1,130.93","1,130.93",979.54,942.72,942.72,942.72,939.12,426.89
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,42.35,36.16,34.55,34.55,34.55,34.28,18.63


In [42]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_BDT_Lxy-based_trained_5to50_Signal_ctau-1_samples_EventCount_Weighted'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDT_Lxy-based_trained_5to50_Signal_ctau-1_samples_EventCount_Weighted.csv


In [43]:
# without m1 = 30 GeV
ct = 1.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,51.58,12.17,12.17,10.29,9.93,9.93,9.93,9.80,2.03
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,"1,627.11",320.24,320.24,273.97,264.43,264.40,264.40,261.91,95.09
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,"2,122.81",408.93,408.93,355.40,344.10,344.10,344.10,341.27,129.49
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,90.58,20.74,20.74,17.64,16.99,16.99,16.99,16.76,5.62
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,"2,928.06",529.56,529.56,449.92,436.79,436.79,436.79,431.62,173.72
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,"4,153.77",742.93,742.93,631.48,605.60,605.60,605.60,598.76,253.16
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,"2,748.24",349.57,349.57,288.31,269.62,268.98,268.98,266.40,114.19
sig_2018_Mchi-60p0_dMchi-20p0_ctau-1,5.68,1.33,1.33,1.10,1.07,1.07,1.07,1.05,0.30
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,"6,562.28","1,130.93","1,130.93",979.54,942.72,942.72,942.72,939.12,426.89
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,205.19,42.35,42.35,36.16,34.55,34.55,34.55,34.28,18.63


In [44]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,4.78,0.92,0.92,0.78,0.75,0.75,0.75,0.73,0.21
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,6.15,1.17,1.17,0.98,0.95,0.95,0.95,0.93,0.35
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,147.91,23.74,23.74,20.09,19.41,19.41,19.41,19.13,7.22
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,197.37,31.54,31.54,26.99,26.02,26.02,26.02,25.80,9.65
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,8.45,1.51,1.51,1.29,1.24,1.24,1.24,1.21,0.45
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,267.87,41.40,41.40,35.62,34.51,34.51,34.51,33.96,13.86
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,390.29,56.79,56.79,48.76,46.63,46.60,46.60,46.02,17.34
sig_2018_Mchi-60p0_dMchi-20p0_ctau-10,0.55,0.10,0.10,0.08,0.08,0.08,0.08,0.07,0.03
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,"6,090.24",583.27,583.27,483.38,451.73,451.73,451.73,439.55,97.39
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,22.98,19.10,18.10,18.03,18.03,16.97,5.14


In [45]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_BDT_Lxy-based_trained_5to50_Signal_ctau-10_samples_EventCount_Weighted'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDT_Lxy-based_trained_5to50_Signal_ctau-10_samples_EventCount_Weighted.csv


In [46]:
# without m1 = 30 GeV
ct = 10.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,4.78,0.92,0.92,0.78,0.75,0.75,0.75,0.73,0.21
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,6.15,1.17,1.17,0.98,0.95,0.95,0.95,0.93,0.35
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,147.91,23.74,23.74,20.09,19.41,19.41,19.41,19.13,7.22
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,197.37,31.54,31.54,26.99,26.02,26.02,26.02,25.80,9.65
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,8.45,1.51,1.51,1.29,1.24,1.24,1.24,1.21,0.45
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,267.87,41.40,41.40,35.62,34.51,34.51,34.51,33.96,13.86
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,390.29,56.79,56.79,48.76,46.63,46.60,46.60,46.02,17.34
sig_2018_Mchi-60p0_dMchi-20p0_ctau-10,0.55,0.10,0.10,0.08,0.08,0.08,0.08,0.07,0.03
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,"6,090.24",583.27,583.27,483.38,451.73,451.73,451.73,439.55,97.39
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,211.62,22.98,22.98,19.10,18.10,18.03,18.03,16.97,5.14


In [47]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,10.55,1.16,1.16,0.98,0.96,0.96,0.96,0.93,0.22
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,0.34,0.04,0.04,0.03,0.03,0.03,0.03,0.03,0.01
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,0.45,0.05,0.05,0.05,0.04,0.04,0.04,0.04,0.01
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,13.98,1.55,1.55,1.31,1.27,1.27,1.27,1.23,0.30
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,0.61,0.07,0.07,0.06,0.05,0.05,0.05,0.05,0.01
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,19.49,2.06,2.06,1.74,1.69,1.69,1.69,1.63,0.35
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,27.17,2.62,2.62,2.21,2.10,2.10,2.10,2.04,0.49
sig_2018_Mchi-60p0_dMchi-20p0_ctau-100,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.25,1.08,1.02,1.02,1.02,0.96,0.09
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,443.66,36.19,36.19,30.61,28.82,28.82,28.82,27.73,2.04


In [48]:
outdir = './csv/Cutflow/EventCount_Weighted/'
outname = 'SR_BDT_Lxy-based_trained_5to50_Signal_ctau-100_samples_EventCount_Weighted'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Weighted//SR_BDT_Lxy-based_trained_5to50_Signal_ctau-100_samples_EventCount_Weighted.csv


In [49]:
# without m1 = 30 GeV
ct = 100.0
m1 = 30.0

points = (si["ctau"] == ct) & (si["m1"] != m1)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,10.55,1.16,1.16,0.98,0.96,0.96,0.96,0.93,0.22
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,0.34,0.04,0.04,0.03,0.03,0.03,0.03,0.03,0.01
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,0.45,0.05,0.05,0.05,0.04,0.04,0.04,0.04,0.01
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,13.98,1.55,1.55,1.31,1.27,1.27,1.27,1.23,0.30
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,0.61,0.07,0.07,0.06,0.05,0.05,0.05,0.05,0.01
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,19.49,2.06,2.06,1.74,1.69,1.69,1.69,1.63,0.35
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,27.17,2.62,2.62,2.21,2.10,2.10,2.10,2.04,0.49
sig_2018_Mchi-60p0_dMchi-20p0_ctau-100,0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,14.29,1.25,1.25,1.08,1.02,1.02,1.02,0.96,0.09
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,443.66,36.19,36.19,30.61,28.82,28.82,28.82,27.73,2.04


### Raw count

In [50]:
pd.options.display.float_format = '{:,.0f}'.format

In [51]:
raw_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_nevts')

In [52]:
table = pd.DataFrame.from_dict(raw_cutflow, orient='index')
table.columns = cutnames

In [53]:
ct = 1.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,31711,7481,7481,6327,6108,6108,6108,6023,1251
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,48974,9639,9639,8246,7959,7958,7958,7883,2862
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,12770,2460,2460,2138,2070,2070,2070,2053,779
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,10607,2429,2429,2066,1990,1990,1990,1963,658
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,13602,2460,2460,2090,2029,2029,2029,2005,807
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,11551,2066,2066,1756,1684,1684,1684,1665,704
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,4261,542,542,447,418,417,417,413,177
sig_2018_Mchi-60p0_dMchi-20p0_ctau-1,7768,1827,1827,1510,1464,1464,1464,1436,408
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,14604,2517,2517,2180,2098,2098,2098,2090,950
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,761,157,157,134,128,128,128,127,69


In [54]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_BDT_Lxy-based_trained_5to50_Signal_ctau-1_samples_EventCount_Raw'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDT_Lxy-based_trained_5to50_Signal_ctau-1_samples_EventCount_Raw.csv


In [55]:
ct = 10.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,31933,6128,6128,5202,5027,5025,5025,4904,1397
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,7516,1436,1436,1195,1163,1163,1163,1132,422
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,32183,5166,5166,4371,4224,4224,4224,4163,1571
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,9944,1589,1589,1360,1311,1311,1311,1300,486
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,10438,1868,1868,1588,1531,1530,1530,1494,551
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,10806,1670,1670,1437,1392,1392,1392,1370,559
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,11566,1683,1683,1445,1382,1381,1381,1364,514
sig_2018_Mchi-60p0_dMchi-20p0_ctau-10,7429,1311,1311,1104,1072,1072,1072,1014,374
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5002,479,479,397,371,371,371,361,80
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,3168,344,344,286,271,270,270,254,77


In [56]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_BDT_Lxy-based_trained_5to50_Signal_ctau-10_samples_EventCount_Raw'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDT_Lxy-based_trained_5to50_Signal_ctau-10_samples_EventCount_Raw.csv


In [57]:
ct = 100.0

points = (si["ctau"] == ct)

df = table.loc[list(si[points].name)]
#df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,25608,2813,2813,2386,2326,2326,2326,2261,544
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,23255,2803,2803,2364,2281,2280,2280,2173,403
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,10603,1240,1240,1062,1023,1022,1022,972,230
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,8905,989,989,834,808,807,807,782,188
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,7894,873,873,731,696,696,696,666,158
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,9055,957,957,810,783,783,783,757,164
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,7990,771,771,650,617,617,617,600,144
sig_2018_Mchi-60p0_dMchi-20p0_ctau-100,5452,602,602,511,498,498,498,457,106
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,2177,190,190,165,155,155,155,147,14
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,10655,869,869,735,692,692,692,666,49


In [58]:
outdir = './csv/Cutflow/EventCount_Raw/'
outname = 'SR_BDT_Lxy-based_trained_5to50_Signal_ctau-100_samples_EventCount_Raw'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname, isSignal=True)

Saved: ./csv/Cutflow/EventCount_Raw//SR_BDT_Lxy-based_trained_5to50_Signal_ctau-100_samples_EventCount_Raw.csv


In [59]:
# for a given ctau and mass splitting
ct = 1.0
delta = 0.1

points = (si["ctau"] == ct) & (si["delta"] == delta)

df = table.loc[list(si[points].name)]
df.loc["Total"] = df.sum()

df

,Preselections,0 < n(jet) < 3 & n(good vertex) > 0,computing BDT score,No b-tagged jets,Leading jet |eta| < 2.4,Leading jet pT > 80 GeV,"dPhi(MET,leading jet) > 1.5","dPhi(MET,all jets) > 0.75",BDT
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,48974,9639,9639,8246,7959,7958,7958,7883,2862
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,12770,2460,2460,2138,2070,2070,2070,2053,779
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,13602,2460,2460,2090,2029,2029,2029,2005,807
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,11551,2066,2066,1756,1684,1684,1684,1665,704
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,14604,2517,2517,2180,2098,2098,2098,2090,950
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,10151,1678,1678,1441,1373,1371,1371,1363,635
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,3558,516,516,456,436,436,436,432,126
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,7390,1090,1090,939,873,872,872,866,339
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,8600,1283,1283,1081,1033,1032,1032,1023,444
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,10056,1508,1508,1291,1248,1248,1248,1237,538


## S/sqrt(B)

In [60]:
nBkg

29.310581931834893

In [61]:
count_cutflow = utils.get_signal_cutflow_dict(sig_histo, 'cutflow_cts')
table = pd.DataFrame.from_dict(count_cutflow, orient='index')
table.columns = cutnames

In [62]:
pd.options.display.float_format = '{:,.4f}'.format

In [63]:
df = pd.DataFrame(table.loc[:, cutnames[-1]]) # last cut

m1_list = []
delta_list = []
ctau_list = []

for point in df.index.values:
    sig_dict = ptools.signalPoint(point)
    m1_list.append(sig_dict['m1'])
    delta_list.append(sig_dict['delta'])
    ctau_list.append(sig_dict['ctau'])

df['m1'] = m1_list
df['delta'] = delta_list
df['ctau'] = ctau_list

df = df.sort_values(by=['m1']) # sort by m1

In [64]:
# Missing rows
#df[ (df['m1'] == 60) & (df['delta'] == 0.2) ]

df.loc[len(df.index)] = [0, 60, 0.2, 1]  
df.loc[len(df.index)] = [0, 60, 0.2, 10]  
df.loc[len(df.index)] = [0, 60, 0.2, 100] 
df.loc[len(df.index)] = [0, 80, 0.2, 1] 

df = df.sort_values(by=['m1']) # sort by m1

In [65]:
df = df.rename(columns={df.columns[0]: 'Signal'})

df['Background'] = nBkg
df

,Signal,m1,delta,ctau,Background
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,114.1922,5.0000,0.2000,1.0000,29.3106
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,0.0918,5.0000,0.2000,100.0000,29.3106
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,97.3914,5.0000,0.1000,10.0000,29.3106
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.1449,5.0000,0.2000,10.0000,29.3106
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,"2,208.5952",5.0000,0.1000,1.0000,29.3106
...,...,...,...,...,...
sig_2018_Mchi-110p0_dMchi-20p0_ctau-100,0.0048,100.0000,0.2000,100.0000,29.3106
sig_2018_Mchi-105p0_dMchi-10p0_ctau-1,70.4692,100.0000,0.1000,1.0000,29.3106
sig_2018_Mchi-110p0_dMchi-20p0_ctau-1,1.3160,100.0000,0.2000,1.0000,29.3106
sig_2018_Mchi-110p0_dMchi-20p0_ctau-10,0.1472,100.0000,0.2000,10.0000,29.3106


In [66]:
df['S/sqrt(B)'] = df['Signal'] / np.sqrt(df['Background'])

df = df[['m1', 'delta', 'ctau', 'Signal', 'Background', 'S/sqrt(B)']]
df

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0000,0.2000,1.0000,114.1922,29.3106,21.0923
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.0000,0.2000,100.0000,0.0918,29.3106,0.0170
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0000,0.1000,10.0000,97.3914,29.3106,17.9891
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0000,0.2000,10.0000,5.1449,29.3106,0.9503
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0000,0.1000,1.0000,"2,208.5952",29.3106,407.9471
...,...,...,...,...,...,...
sig_2018_Mchi-110p0_dMchi-20p0_ctau-100,100.0000,0.2000,100.0000,0.0048,29.3106,0.0009
sig_2018_Mchi-105p0_dMchi-10p0_ctau-1,100.0000,0.1000,1.0000,70.4692,29.3106,13.0163
sig_2018_Mchi-110p0_dMchi-20p0_ctau-1,100.0000,0.2000,1.0000,1.3160,29.3106,0.2431
sig_2018_Mchi-110p0_dMchi-20p0_ctau-10,100.0000,0.2000,10.0000,0.1472,29.3106,0.0272


In [67]:
outdir = './csv/SR_Yields/'
outname = 'SR_BDT_Lxy-based_trained_5to50_S_over_B'

df_to_save = df

save_df_to_csv(df_to_save, outdir, outname)

Saved: ./csv/SR_Yields//SR_BDT_Lxy-based_trained_5to50_S_over_B.csv


### ctau = 1mm signal points

In [68]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 1mm, delta = 0.1
ct = 1.0
delta = 0.1

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-1,5.0,0.1,1.0,"2,208.5952",29.3106,407.9471
sig_2018_Mchi-10p5_dMchi-1p0_ctau-1,10.0,0.1,1.0,"1,807.7566",29.3106,333.9087
sig_2018_Mchi-21p0_dMchi-2p0_ctau-1,20.0,0.1,1.0,"1,567.9834",29.3106,289.6205
sig_2018_Mchi-31p5_dMchi-3p0_ctau-1,30.0,0.1,1.0,"1,807.8675",29.3106,333.9292
sig_2018_Mchi-42p0_dMchi-4p0_ctau-1,40.0,0.1,1.0,"1,013.0309",29.3106,187.1158
sig_2018_Mchi-52p5_dMchi-5p0_ctau-1,50.0,0.1,1.0,426.8943,29.3106,78.8512
sig_2018_Mchi-63p0_dMchi-6p0_ctau-1,60.0,0.1,1.0,253.1563,29.3106,46.7602
sig_2018_Mchi-73p5_dMchi-7p0_ctau-1,70.0,0.1,1.0,173.7176,29.3106,32.0872
sig_2018_Mchi-84p0_dMchi-8p0_ctau-1,80.0,0.1,1.0,129.4925,29.3106,23.9184
sig_2018_Mchi-94p5_dMchi-9p0_ctau-1,90.0,0.1,1.0,95.0891,29.3106,17.5638


In [69]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 1mm, delta = 0.2
ct = 1.0
delta = 0.2

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-1,5.0,0.2,1.0,114.1922,29.3106,21.0923
sig_2018_Mchi-11p0_dMchi-2p0_ctau-1,10.0,0.2,1.0,80.2282,29.3106,14.8189
sig_2018_Mchi-22p0_dMchi-4p0_ctau-1,20.0,0.2,1.0,71.6514,29.3106,13.2347
sig_2018_Mchi-33p0_dMchi-6p0_ctau-1,30.0,0.2,1.0,"4,184.5531",29.3106,772.9241
sig_2018_Mchi-44p0_dMchi-8p0_ctau-1,40.0,0.2,1.0,40.5178,29.3106,7.4840
sig_2018_Mchi-55p0_dMchi-10p0_ctau-1,50.0,0.2,1.0,18.6272,29.3106,3.4406
65,60.0,0.2,1.0,0.0000,29.3106,0.0000
sig_2018_Mchi-77p0_dMchi-14p0_ctau-1,70.0,0.2,1.0,5.6189,29.3106,1.0379
68,80.0,0.2,1.0,0.0000,29.3106,0.0000
sig_2018_Mchi-99p0_dMchi-18p0_ctau-1,90.0,0.2,1.0,2.0348,29.3106,0.3758


### ctau = 10mm signal points

In [70]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 10mm, delta = 0.1
ct = 10.0
delta = 0.1

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-10,5.0,0.1,10.0,97.3914,29.3106,17.9891
sig_2018_Mchi-10p5_dMchi-1p0_ctau-10,10.0,0.1,10.0,96.9867,29.3106,17.9143
sig_2018_Mchi-21p0_dMchi-2p0_ctau-10,20.0,0.1,10.0,107.9054,29.3106,19.9311
sig_2018_Mchi-31p5_dMchi-3p0_ctau-10,30.0,0.1,10.0,"1,237.5039",29.3106,228.5780
sig_2018_Mchi-42p0_dMchi-4p0_ctau-10,40.0,0.1,10.0,74.4949,29.3106,13.7599
sig_2018_Mchi-52p5_dMchi-5p0_ctau-10,50.0,0.1,10.0,29.8376,29.3106,5.5113
sig_2018_Mchi-63p0_dMchi-6p0_ctau-10,60.0,0.1,10.0,17.3425,29.3106,3.2033
sig_2018_Mchi-73p5_dMchi-7p0_ctau-10,70.0,0.1,10.0,13.8576,29.3106,2.5596
sig_2018_Mchi-84p0_dMchi-8p0_ctau-10,80.0,0.1,10.0,9.6478,29.3106,1.7820
sig_2018_Mchi-94p5_dMchi-9p0_ctau-10,90.0,0.1,10.0,7.2199,29.3106,1.3336


In [71]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 10mm, delta = 0.2
ct = 10.0
delta = 0.2

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-10,5.0,0.2,10.0,5.1449,29.3106,0.9503
sig_2018_Mchi-11p0_dMchi-2p0_ctau-10,10.0,0.2,10.0,3.7844,29.3106,0.6990
sig_2018_Mchi-22p0_dMchi-4p0_ctau-10,20.0,0.2,10.0,4.2231,29.3106,0.7800
sig_2018_Mchi-33p0_dMchi-6p0_ctau-10,30.0,0.2,10.0,857.9306,29.3106,158.4674
sig_2018_Mchi-44p0_dMchi-8p0_ctau-10,40.0,0.2,10.0,2.7392,29.3106,0.5060
sig_2018_Mchi-55p0_dMchi-10p0_ctau-10,50.0,0.2,10.0,1.3188,29.3106,0.2436
66,60.0,0.2,10.0,0.0000,29.3106,0.0000
sig_2018_Mchi-77p0_dMchi-14p0_ctau-10,70.0,0.2,10.0,0.4460,29.3106,0.0824
sig_2018_Mchi-88p0_dMchi-16p0_ctau-10,80.0,0.2,10.0,0.3453,29.3106,0.0638
sig_2018_Mchi-99p0_dMchi-18p0_ctau-10,90.0,0.2,10.0,0.2090,29.3106,0.0386


### ctau = 100mm signal points

In [72]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 100mm, delta = 0.1
ct = 100.0
delta = 0.1

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p25_dMchi-0p5_ctau-100,5.0,0.1,100.0,2.0411,29.3106,0.3770
sig_2018_Mchi-10p5_dMchi-1p0_ctau-100,10.0,0.1,100.0,1.9310,29.3106,0.3567
sig_2018_Mchi-21p0_dMchi-2p0_ctau-100,20.0,0.1,100.0,2.0847,29.3106,0.3851
sig_2018_Mchi-31p5_dMchi-3p0_ctau-100,30.0,0.1,100.0,317.7761,29.3106,58.6961
sig_2018_Mchi-42p0_dMchi-4p0_ctau-100,40.0,0.1,100.0,1.9249,29.3106,0.3555
sig_2018_Mchi-52p5_dMchi-5p0_ctau-100,50.0,0.1,100.0,0.8313,29.3106,0.1535
sig_2018_Mchi-63p0_dMchi-6p0_ctau-100,60.0,0.1,100.0,0.4898,29.3106,0.0905
sig_2018_Mchi-73p5_dMchi-7p0_ctau-100,70.0,0.1,100.0,0.3531,29.3106,0.0652
sig_2018_Mchi-84p0_dMchi-8p0_ctau-100,80.0,0.1,100.0,0.2950,29.3106,0.0545
sig_2018_Mchi-94p5_dMchi-9p0_ctau-100,90.0,0.1,100.0,0.2241,29.3106,0.0414


In [73]:
pd.options.display.float_format = '{:,.4f}'.format

# S/sqrt(B) in ctau = 100mm, delta = 0.2
ct = 100.0
delta = 0.2

df_to_show = df[ (df.ctau == ct) & (df.delta == delta) ]
df_to_show.loc[:, 'm1'] = df_to_show.loc[:, 'm1'].map('{:,.1f}'.format)
df_to_show.loc[:, 'delta'] = df_to_show.loc[:, 'delta'].map('{:,.1f}'.format)
df_to_show.loc[:, 'ctau'] = df_to_show.loc[:, 'ctau'].map('{:,.1f}'.format)

df_to_show

,m1,delta,ctau,Signal,Background,S/sqrt(B)
sig_2018_Mchi-5p5_dMchi-1p0_ctau-100,5.0,0.2,100.0,0.0918,29.3106,0.0170
sig_2018_Mchi-11p0_dMchi-2p0_ctau-100,10.0,0.2,100.0,0.0690,29.3106,0.0127
sig_2018_Mchi-22p0_dMchi-4p0_ctau-100,20.0,0.2,100.0,0.1024,29.3106,0.0189
sig_2018_Mchi-33p0_dMchi-6p0_ctau-100,30.0,0.2,100.0,23.6162,29.3106,4.3621
sig_2018_Mchi-44p0_dMchi-8p0_ctau-100,40.0,0.2,100.0,0.0694,29.3106,0.0128
sig_2018_Mchi-55p0_dMchi-10p0_ctau-100,50.0,0.2,100.0,0.0317,29.3106,0.0059
67,60.0,0.2,100.0,0.0000,29.3106,0.0000
sig_2018_Mchi-77p0_dMchi-14p0_ctau-100,70.0,0.2,100.0,0.0121,29.3106,0.0022
sig_2018_Mchi-88p0_dMchi-16p0_ctau-100,80.0,0.2,100.0,0.0098,29.3106,0.0018
sig_2018_Mchi-99p0_dMchi-18p0_ctau-100,90.0,0.2,100.0,0.0059,29.3106,0.0011
